In [4]:
import re
import numpy as np

def read_pgm(filename, byteorder='>'):
    """Return image data from a raw PGM file as np array.

    Format specification: http://netpbm.sourceforge.net/doc/pgm.html

    """
    with open(filename, 'rb') as f:
        buffer = f.read()
    try:
        header, width, height, maxval = re.search(
            b"(^P5\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    except AttributeError:
        raise ValueError("Not a raw PGM file: '%s'" % filename)
    return np.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))

def resizeImage(image, newSize):
    a = image.shape[0]
    b = newSize
    ratio = a/b
    resizeAToB= np.zeros((a,b))
    for i in range(a):
        for j in range(b): 
            if i//ratio == j:
                resizeAToB[i,j] = 1. / ratio
    return np.matmul(resizeAToB.T, np.matmul(image, resizeAToB))

In [5]:
from subprocess import Popen, PIPE

def flatten_list_of_strings(args):
    return reduce(lambda x,y: x +' '+y, args)

def run_exp(input_image_path, discretization_n, amount_of_rays, random_noise, exp_list):
    tp_exec = '../build/tp3'
    exp_name = flatten_list_of_strings(exp_list)
    
    output_image_name = "{}_{}_celdas_{}_ruido_{}_rayos".format(
        exp_name, discretization_n, random_noise, amount_of_rays)
    output_image_dir = "../results/"
    output_image_path = output_image_dir + "\"" + output_image_name + '.pgm\"'
    output_image_path = output_image_path.replace(' ', '_')
    
    args = [tp_exec, "-i", str(input_image_path), "-o", str(output_image_path), 
            "-n", str(discretization_n), "-r", str(random_noise)]
    for exp in exp_list:
        if 'g15' in exp or 'g14' in exp:
            args.extend(exp.split())
            args.append(str(amount_of_rays//len(exp_list)))
        else:
            args.extend([exp, str(amount_of_rays//len(exp_list))])
        
    print(args)
    print(flatten_list_of_strings(args))
    
    run_logs = {}
    p = Popen(args, stdout=PIPE, bufsize=1)
    with p.stdout:
        for line in iter(p.stdout.readline, b''):
            try:
                key, val = line[:-1].split(': ')
                if key in ['error','debug']:
                    print(line)
                else:
                    run_logs[key] = val    
            except:
                print('COMANDO FEO: {}'.format(line))
    p.wait() # wait for the subprocess to exit
    print(output_image_path)
    output_image = read_pgm(output_image_path)  # .reshape(-1)
    process_data = {'discretization_n':float(discretization_n), 
            'random_noise': float(random_noise),
            'rayos': float(amount_of_rays),
            'image': output_image,
            'exp_name': exp_name,
            'output_image_name': output_image_name}
    
    for key in ['tiempo_cml_microseconds', 'tiempo_geometria_microseconds', 'tiempo_matrices_microseconds']:
        run_logs[key] = float(run_logs[key])
    
    return dict(run_logs, **process_data)

def print_run_logs(run_logs_and_process_data):
    for key in ['tiempo_cml_microseconds', 'tiempo_geometria_microseconds', 'tiempo_matrices_microseconds']:
        print('{} : {}'.format(key, run_logs_and_process_data[key]))

#### Run exp

In [6]:
input_image_path = "../imgs_TC/tomo.pgm"
divisors_of_100 = [25]
import pandas as pd
output = []
for div in divisors_of_100:
    exp_names = [['-g4']]
    exp_names.extend([['-g14b '+focos+' -g14a'] for focos in ['50']])
    for exp in exp_names:
        for random in [0]:        
            for amount_of_rays in [500]:
                output.append(run_exp(input_image_path, div, amount_of_rays, random, exp))
                
                print(print_run_logs(output[-1])) # print time taken

outputs_df = pd.DataFrame(output)
outputs_df.head()

['../build/tp3', '-i', '../imgs_TC/tomo.pgm', '-o', '../results/"-g4_25_celdas_0_ruido_500_rayos.pgm"', '-n', '25', '-r', '0', '-g4', '500']
../build/tp3 -i ../imgs_TC/tomo.pgm -o ../results/"-g4_25_celdas_0_ruido_500_rayos.pgm" -n 25 -r 0 -g4 500
COMANDO FEO: ejecutar_analisis_tomografico

debug: _rayos.size() 500

COMANDO FEO: matrices creadas

debug: Calculando autovector nº 1 / 625

debug: Calculando autovector nº 2 / 625

debug: Calculando autovector nº 3 / 625

debug: Calculando autovector nº 4 / 625

debug: Calculando autovector nº 5 / 625

debug: Calculando autovector nº 6 / 625

debug: Calculando autovector nº 7 / 625

debug: Calculando autovector nº 8 / 625

debug: Calculando autovector nº 9 / 625

debug: Calculando autovector nº 10 / 625

debug: Calculando autovector nº 11 / 625

debug: Calculando autovector nº 12 / 625

debug: Calculando autovector nº 13 / 625

debug: Calculando autovector nº 14 / 625

debug: Calculando autovector nº 15 / 625

debug: Calculando autovector n


debug: Calculando autovector nº 190 / 625

debug: Calculando autovector nº 191 / 625

debug: Calculando autovector nº 192 / 625

debug: Calculando autovector nº 193 / 625

debug: Calculando autovector nº 194 / 625

debug: Calculando autovector nº 195 / 625

debug: Calculando autovector nº 196 / 625

debug: Calculando autovector nº 197 / 625

debug: Calculando autovector nº 198 / 625

debug: Calculando autovector nº 199 / 625

debug: Calculando autovector nº 200 / 625

debug: Calculando autovector nº 201 / 625

debug: Calculando autovector nº 202 / 625

debug: Calculando autovector nº 203 / 625

debug: Calculando autovector nº 204 / 625

debug: Calculando autovector nº 205 / 625

debug: Calculando autovector nº 206 / 625

debug: Calculando autovector nº 207 / 625

debug: Calculando autovector nº 208 / 625

debug: Calculando autovector nº 209 / 625

debug: Calculando autovector nº 210 / 625

debug: Calculando autovector nº 211 / 625

debug: Calculando autovector nº 212 / 625

debug: Cal

debug: Calculando autovector nº 386 / 625

debug: Calculando autovector nº 387 / 625

debug: Calculando autovector nº 388 / 625

debug: Calculando autovector nº 389 / 625

debug: Calculando autovector nº 390 / 625

debug: Calculando autovector nº 391 / 625

debug: Calculando autovector nº 392 / 625

debug: Calculando autovector nº 393 / 625

debug: Calculando autovector nº 394 / 625

debug: Calculando autovector nº 395 / 625

debug: Calculando autovector nº 396 / 625

debug: Calculando autovector nº 397 / 625

debug: Calculando autovector nº 398 / 625

debug: Calculando autovector nº 399 / 625

debug: Calculando autovector nº 400 / 625

debug: Calculando autovector nº 401 / 625

debug: Calculando autovector nº 402 / 625

debug: Calculando autovector nº 403 / 625

debug: Calculando autovector nº 404 / 625

debug: Calculando autovector nº 405 / 625

debug: Calculando autovector nº 406 / 625

debug: Calculando autovector nº 407 / 625

debug: Calculando autovector nº 408 / 625

debug: Calc

debug: Calculando autovector nº 584 / 625

debug: Calculando autovector nº 585 / 625

debug: Calculando autovector nº 586 / 625

debug: Calculando autovector nº 587 / 625

debug: Calculando autovector nº 588 / 625

debug: Calculando autovector nº 589 / 625

debug: Calculando autovector nº 590 / 625

debug: Calculando autovector nº 591 / 625

debug: Calculando autovector nº 592 / 625

debug: Calculando autovector nº 593 / 625

debug: Calculando autovector nº 594 / 625

debug: Calculando autovector nº 595 / 625

debug: Calculando autovector nº 596 / 625

debug: Calculando autovector nº 597 / 625

debug: Calculando autovector nº 598 / 625

debug: Calculando autovector nº 599 / 625

debug: Calculando autovector nº 600 / 625

debug: Calculando autovector nº 601 / 625

debug: Calculando autovector nº 602 / 625

debug: Calculando autovector nº 603 / 625

debug: Calculando autovector nº 604 / 625

debug: Calculando autovector nº 605 / 625

debug: Calculando autovector nº 606 / 625

debug: Calc

KeyError: 'tiempo_cml_microseconds'

#### Input images

In [ ]:
from matplotlib import pyplot 
input_image_path = "../imgs_TC/tomo.pgm"
input_image = read_pgm(input_image_path, byteorder='<')
divisors = [20,25,50]
resized_input_images = {div: resizeImage(input_image, div) for div in divisors}
for div in divisors:    
    resized_input_image = resized_input_images[div]
    pyplot.imshow(resized_input_image, pyplot.cm.gray)
    pyplot.savefig('./resized_tomo{}x{}.png'.format(div,div))
    pyplot.show()
    

#### Reconstructed images

In [ ]:
image_and_metadata_headers = ['image', 'output_image_name']
for i,row in outputs_df[image_and_metadata_headers].iterrows():
    output_image_name = row['output_image_name']
    pyplot.title(output_image_name)
    pyplot.imshow(row['image'], pyplot.cm.gray)
    pyplot.show()



#### Compute mean squared error

In [ ]:
from sklearn.metrics import mean_squared_error
def mse_vs_benchmark(resized_input_images, output_img):
    output_img = output_img.reshape(-1)
    div = np.sqrt(len(output_img))
    resized_input_image = resized_input_images[div].reshape(-1)
    return mean_squared_error(resized_input_image, output_img)

from functools import partial
mse_vs_resized_input_images = partial(mse_vs_benchmark,resized_input_images)

outputs_df['mse_vs_benchmark'] = outputs_df['image'].map(mse_vs_resized_input_images)
outputs_df.head()

In [ ]:
from textwrap import wrap
def plot_param_vs_mse_per_exp(outputs_df, param_key):
    print('{} vs {} plots'.format(param_key, 'mse_vs_benchmark'))
    for exp_name in set(outputs_df['exp_name']):
        exp_filtered = outputs_df[(outputs_df['exp_name'] == exp_name)]

        exp_filtered = exp_filtered.groupby([param_key],as_index=False)['mse_vs_benchmark'].mean()

        title = 'exp_name_{}_{}_vs_{}'.format(exp_name,param_key, 'mse_vs_benchmark')
        title = "\n".join(wrap(title, 30))
        exp_filtered.plot(param_key, 'mse_vs_benchmark',title=title)

def plot_param_vs_times_per_exp(outputs_df, param_key):
    time_keys = ['tiempo_cml_microseconds', 'tiempo_geometria_microseconds', 'tiempo_matrices_microseconds']
    print('{} vs time plots'.format(param_key))
    for exp_name in set(outputs_df['exp_name']):
        exp_filtered = outputs_df[(outputs_df['exp_name'] == exp_name)]
        exp_filtered = exp_filtered.groupby([param_key],as_index=False)[time_keys].mean()

        title = 'exp_name_{}_{}_vs_time'.format(exp_name,param_key)
        title = "\n".join(wrap(title, 30))
        exp_filtered.plot(param_key, time_keys,title=title)

plot_param_vs_times_per_exp(outputs_df, 'rayos')

In [ ]:
plot_param_vs_mse_per_exp(outputs_df, 'rayos')
plot_param_vs_mse_per_exp(outputs_df, 'discretization_n')
plot_param_vs_mse_per_exp(outputs_df, 'random_noise')

In [ ]:
pd.options.mode.chained_assignment = None
from textwrap import wrap
time_keys = ['tiempo_cml_microseconds', 'tiempo_geometria_microseconds', 'tiempo_matrices_microseconds']

for exp_name in set(outputs_df['exp_name']):
    exp_filtered = outputs_df[(outputs_df['exp_name'] == exp_name)]
    exp_filtered['time'] = exp_filtered['tiempo_cml_microseconds'] + exp_filtered['tiempo_geometria_microseconds'] + exp_filtered['tiempo_matrices_microseconds']
    title =  '{}_time_and_noise_vs_mse'.format(exp_name)
    title = "\n".join(wrap(title, 30))
    fig, ax = plt.subplots()
    exp_filtered.plot.scatter(x='rayos',y='random_noise', c='mse_vs_benchmark',title=title, legend=True, cmap="cool",ax=ax);
